### 為了能夠提供更好的疾病詢答服務給一般民眾，因此你奉命要建立一個自動問答機器人。為了能夠正確回覆使用者的問題，我們首先從疾管署的官網上(http://www.cdc.gov.tw/disease.aspx?treeid=8d54c504e820735b )的傳染病專區蒐集了所有「疾病名稱」 、「相關發病症狀」與「治療方法與就醫資訊」，並將該資料存放於(https://raw.githubusercontent.com/ywchiu/cdc_course/master/data/disease_info.xlsx)　，現在有一民眾在網路問了以下問題：「請問陳醫生，我們家三歲小孩發燒，且嘴巴周圍和手腳都出現水泡和潰瘍，八歲的姊姊發燒喉嚨痛、手腳長水泡，請問我們家小孩生了什麼病呢？」，請試著用傳染病專區資料所建立的問答機器人回答以下問題：

1. 請寫一個函數getResponse，當使用者輸入問題時，機器人可以根據問題與「相關發病症狀」的相似度，回傳最相似疾病症狀的「疾病名稱」與「治療方法與就醫資訊」
2. 回答與該問題最相關的「疾病名稱」與「治療方法與就醫資訊」

## 問答資料抓取

In [2]:
import requests
res = requests.get('http://www.cdc.gov.tw/disease.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b')
#res.text

In [9]:
from bs4 import BeautifulSoup
domain = 'http://www.cdc.gov.tw/'
soup = BeautifulSoup(res.text, 'html.parser')
for li in soup.select('.list_content td'):
    if len(li.select('a')) >0:
        print(domain + li.select('a')[0]['href'])

http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=77BFF3D4F9CB7982
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=59FB45DE0CD3F8A0
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=AC308F5F5E7F4299
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=CFB77829C7ABD761
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=8F87A67697D2278B
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=5976EE7146BF5A34
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=1C7BF6D5F52F478C
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=3C994A4FA28416FC
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=778876767EA9B06C
h

## 抓取內容頁

In [11]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=77BFF3D4F9CB7982')
soup = BeautifulSoup(res.text, 'html.parser')

In [19]:
disease_dic = {}
for in_p in soup.select('.in_p'):
    h2 = in_p.select('h2')[0].text.strip().strip('：')
    p =  in_p.select('p')[0].text.strip()
    disease_dic[h2] = p


In [20]:
disease_dic

{'傳播方式': '臺灣主要傳播登革熱的病媒蚊為埃及斑蚊（Aedes aegypti）及白線斑蚊（Aedes albopictus），這些蚊子的特徵身體是黑色的，腳上有白斑。其中埃及斑蚊的胸部兩側具有一對似七弦琴的縱線及中間一對黃色的縱線，喜歡棲息於室內的人工容器，或是人為所造成積水的地方；白線斑蚊則是中胸楯板部位中間，有一條白色且明顯的縱紋，並比較喜歡棲息於室外。一天叮咬人的高峰期約在日出後的1-2小時及日落前的2-3小時，此時到戶外活動請做好自我保護措施。',
 '治療方法與就醫資訊': '由於目前沒有特效藥物可治療登革熱，所以感染登革熱的患者，一定要聽從醫師的囑咐，多休息、多喝水、適時服用退燒藥，通常在感染後兩週左右可自行痊癒。此外，對於登革熱重症病患應安排住院，適時的介入措施，提供完整嚴密及持續的照護，可將死亡率從20%以上降到1%以下。',
 '潛伏期': '典型登革熱的潛伏期約為3至8天(最長可達14天)。病人發病前一天至發病後5天的這段期間，稱為「可感染期」，或稱為「病毒血症期」，因病毒存在血液中，如果感染者在這個時期被斑蚊叮咬，那麼這隻斑蚊將感染登革病毒，病毒在蚊子體內經過8-12天的增殖，這隻斑蚊就具有終生傳染病毒的能力，當牠再叮咬其他人時，就會把體內的登革病毒傳染給另一個人。',
 '發病症狀': '每個人的體質不一樣，有些人感染登革熱時，症狀輕微，甚至不會出現生病症狀。而典型登革熱的症狀則是會有突發性的高燒（≧38℃），頭痛、後眼窩痛、肌肉痛、關節痛及出疹等現象；然而，若是先後感染不同型別之登革病毒，有更高機率導致較嚴重的臨床症狀，如果沒有及時就醫或治療，死亡率可以高達20%以上，所以民眾千萬不能掉以輕心！發病後的第3~5天，若病情突然加劇，如發生劇烈疼痛、抽搐、昏迷、意識狀況及血壓改變等，須注意是否進展為登革熱重症。',
 '預防接種建議': '',
 '預防方法': '登革熱是一種「社區病」、「環境病」，且病媒蚊對於叮咬對象並無選擇性，一旦有登革病毒進入社區，且生活周圍有病媒蚊孳生源的環境，就有登革熱流行的可能性，所以民眾平時應做好病媒蚊孳生源的清除工作。此外，民眾平時也應提高警覺，了解登革熱的症狀，除了發病時可及早就醫、早期診斷且適當治療，亦應同時避免再被病媒蚊叮咬，以減少登革病毒再傳播的可能。'}

In [21]:
import requests
from bs4 import BeautifulSoup
def getDiseaseDetail(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    disease_dic = {}
    for in_p in soup.select('.in_p'):
        h2 = in_p.select('h2')[0].text.strip().strip('：')
        p =  in_p.select('p')[0].text.strip()
        disease_dic[h2] = p
    return disease_dic

In [22]:
getDiseaseDetail('http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=8F87A67697D2278B')

{'傳播方式': '腺鼠疫主要由被感染跳蚤（尤其是Xenopsylla cheopis印度鼠蚤）叮咬吸取人類血液或是人類處理被感染動物（尤其老鼠和家兔）或鼠疫感染者屍體的組織時，不慎接觸膿液而感染。肺鼠疫和咽鼠疫藉空氣散播，人類因吸入帶有致病原之飛沫（原發性肺鼠疫病人或已發展出末期鼠疫肺炎的腺鼠疫病人之飛沫）而感染。',
 '治療方法與就醫資訊': '及早使用抗生素治療鼠疫，可顯著降低致死率。當身體出現發燒、寒顫、頭痛、局部淋巴腺腫、虛脫及白血球增多等症狀，並有鼠疫流行地區旅遊史且曾接觸齧齒類動物或遭跳蚤叮咬時，請儘速就醫並主動告知醫師相關動物接觸史及旅遊史等訊息，以及早獲得妥適的治療。',
 '潛伏期': '通常為1～7天。原發性肺鼠疫為1～4天。',
 '發病症狀': '出現發燒、寒顫、頭痛、不適、虛脫及白血球增多，且伴隨下列一種（含）以上之主要臨床型態：',
 '預防接種建議': '死菌疫苗主動免疫可提供數月防護力，適用於高發病率地區的居民、旅客、處理鼠疫桿菌或被感染動物的實驗室人員或防疫人員，且須與其他防護方法一併使用。',
 '預防方法': '最主要為避免被跳蚤叮咬，直接接觸具感染性之組織及暴露於肺鼠疫病人活動地區。'}

## 完成疾病資訊抓取

In [27]:
import requests
from bs4 import BeautifulSoup

domain = 'http://www.cdc.gov.tw/'

def getDiseaseDetail(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    disease_dic = {}
    print(url)
    for in_p in soup.select('.in_p'):
        if (len(in_p.select('h2')) > 0) and (len(in_p.select('p')) > 0 ):
            h2 = in_p.select('h2')[0].text.strip().strip('：')
            p =  in_p.select('p')[0].text.strip()
            disease_dic[h2] = p
    return disease_dic

res = requests.get('http://www.cdc.gov.tw/disease.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b')
soup = BeautifulSoup(res.text, 'html.parser')

disease_info = []
for li in soup.select('.list_content td'):
    if len(li.select('a')) >0:
        link = domain + li.select('a')[0]['href']
        disease_info.append(getDiseaseDetail(link))
        

http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=77BFF3D4F9CB7982
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=59FB45DE0CD3F8A0
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=AC308F5F5E7F4299
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=CFB77829C7ABD761
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=8F87A67697D2278B
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=5976EE7146BF5A34
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=1C7BF6D5F52F478C
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=3C994A4FA28416FC
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=778876767EA9B06C
h

http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=110FA1639DEF7957
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=37DAE22507FE1341
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=B7A99E685C850D64
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=4DC6CDD1D4B8972D
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=4E9C3E6AC79BFB9D
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=93A90D21A8A6D479
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=9618DBD92CAD7F8F
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=D40B2E6AC7484C3B
http://www.cdc.gov.tw/diseaseinfo.aspx?treeid=8d54c504e820735b&nowtreeid=dec84a2f0c6fac5b&tid=F81306B23916A286
h

In [29]:
import pandas
disease_df = pandas.DataFrame(disease_info)
disease_df.head()

,傳播方式,治療方法與就醫資訊,潛伏期,發病症狀,預防接種建議,預防方法
0,臺灣主要傳播登革熱的病媒蚊為埃及斑蚊（Aedes aegypti）及白線斑蚊（Aedes a...,由於目前沒有特效藥物可治療登革熱，所以感染登革熱的患者，一定要聽從醫師的囑咐，多休息、多喝水...,典型登革熱的潛伏期約為3至8天(最長可達14天)。病人發病前一天至發病後5天的這段期間，稱為...,每個人的體質不一樣，有些人感染登革熱時，症狀輕微，甚至不會出現生病症狀。而典型登革熱的症狀則...,,登革熱是一種「社區病」、「環境病」，且病媒蚊對於叮咬對象並無選擇性，一旦有登革病毒進入社區，...
1,屈公病是人被帶有屈公病毒的病媒蚊叮咬而感染，發病前2天至發病後5天間，病人血液中有病毒活動，...,目前沒有治療屈公病毒的藥物，主要是依病情給予支持性療法，感染屈公病患者應聽從醫師囑咐，多休息...,潛伏期2～12天，通常為3～7天。,感染屈公病主要的症狀包括突然發燒、關節疼痛或關節炎、頭痛、噁心、疲倦、肌肉疼痛，約半數的病患...,屈公病目前尚無可施打之疫苗，民眾應加強自身的防蚊措施，並積極清除積水容器，杜絕病媒蚊孳生才是...,屈公病是經由病媒蚊叮咬而傳播的急性傳染病，且病媒蚊對於叮咬對象並無選擇性，一旦有屈公病毒進入...
2,臺灣地區主要病媒蚊是矮小瘧蚊，當被感染且具傳染能力的瘧蚊叮咬人時，將瘧原蟲注入人體，導致感染...,因瘧原蟲潛伏期可能長達數月或數年之久，若曾至瘧疾流行地區旅遊，旅遊期間或旅遊後，出現間歇性發...,大多數人感染後，隨瘧原蟲種類不同而有差異，症狀可能於7天至30天內出現。感染惡性瘧原蟲的潛伏...,感染瘧疾早期的症狀與流感類似，最主要的症狀為發燒、畏寒及顫抖，接著冒冷汗。也可能出現其他症狀...,瘧疾疫苗目前正在研發中，尚未核准上市。,（一） 出國前請先了解瘧疾感染危險地區。世界衛生組織(WHO)公布，目前仍有91個國家為瘧疾...
3,日本腦炎是由病媒蚊傳播的疾病，人不會直接傳染給人。臺灣以三斑家蚊、環蚊家蚊及白頭家蚊為主要的...,目前沒有特效藥物可治療日本腦炎，採取支持性療法。,通常為5至15天。,感染日本腦炎病毒大部分無明顯症狀，約有小於1%的感染者會出現臨床症狀，最常見的臨床表現是急性腦炎。,一、臺灣自1960年代推動日本腦炎預防接種，使用的是以鼠腦製程之不活化日本腦炎疫苗（inac...,一、依規定時程接種日本腦炎疫苗。
4,腺鼠疫主要由被感染跳蚤（尤其是Xenopsylla cheopis印度鼠蚤）叮咬吸取人類血液...,及早使用抗生素治療鼠疫，可顯著降低致死率。當身體出現發燒、寒顫、頭痛、局部淋巴腺腫、虛脫及白...,通常為1～7天。原發性肺鼠疫為1～4天。,出現發燒、寒顫、頭痛、不適、虛脫及白血球增多，且伴隨下列一種（含）以上之主要臨床型態：,死菌疫苗主動免疫可提供數月防護力，適用於高發病率地區的居民、旅客、處理鼠疫桿菌或被感染動物的...,最主要為避免被跳蚤叮咬，直接接觸具感染性之組織及暴露於肺鼠疫病人活動地區。


In [31]:
disease_df.to_excel('disease_info.xlsx')